In [ ]:
pip install -U langchain langchain-community langchain-core langchain-classic langchain_chroma langchain-huggingface langchain-google-genai langchain-groq

In [2]:
import os
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
vector_db = Chroma(
    persist_directory="my_vectordb", 
    embedding_function=embeddings,
    collection_name="office_docs"
)

In [ ]:
os.environ["GROQ_API_KEY"] = "key"

In [6]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [7]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [8]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood, "
    "If user can answer the user Reminder points"
)

In [9]:
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"), # Dynamic history goes here
    ("human", "{input}"),
])

In [10]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [11]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [12]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [13]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [14]:
store = {}

In [15]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    
    # TRIMMING: Keep only last 4 messages to stay efficient
    if len(store[session_id].messages) > 4:
        store[session_id].messages = store[session_id].messages[-4:]
    return store[session_id]

In [16]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [17]:
user_1_config = {"configurable": {"session_id": "user_123"}}
user_2_config = {"configurable": {"session_id": "user_124"}}

In [18]:
res1 = conversational_rag_chain.invoke({"input": "What is the office location?"}, config=user_1_config)
print(f"User 1: What is the office location?\nAI: {res1['answer']}\n")

User 1: What is the office location?
AI: Our office is located in Bangalore.



In [19]:
res2 = conversational_rag_chain.invoke({"input": "What country is that city in?"}, config=user_1_config)
print(f"User 1: What country is that city in?\nAI: {res2['answer']}")

User 1: What country is that city in?
AI: Bangalore is a city in India.


In [20]:
res3 = conversational_rag_chain.invoke({"input": "Remember this Number - 3"}, config=user_1_config)
print(f"User 1: Remember this Number\nAI: {res3['answer']}")

User 1: Remember this Number
AI: I've remembered the number: 3.


In [21]:
res4 = conversational_rag_chain.invoke({"input": "What number I asked you to remember?"}, config=user_1_config)
print(f"User 1: What number I asked you to remember?\nAI: {res4['answer']}")

User 1: What number I asked you to remember?
AI: You asked me to remember the number 3.


In [22]:
res5 = conversational_rag_chain.invoke({"input": "What number I asked you to remember?"}, config=user_2_config)
print(f"User 2: What number I asked you to remember?\nAI: {res5['answer']}")

User 2: What number I asked you to remember?
AI: You didn't ask me to remember any number. Our conversation just started with you providing some context about our office being in Bangalore and the CEO being Sarah Jenkins.


In [23]:
store

{'user_123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What country is that city in?', additional_kwargs={}, response_metadata={}), AIMessage(content='Bangalore is a city in India.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='Remember this Number - 3', additional_kwargs={}, response_metadata={}), AIMessage(content="I've remembered the number: 3.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='What number I asked you to remember?', additional_kwargs={}, response_metadata={}), AIMessage(content='You asked me to remember the number 3.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]),
 'user_124': InMemoryChatMessageHistory(messages=[HumanMessage(content='What number I asked you to remember?', additional_kwargs={}, response_metadata={}), AIMessage(content="You didn't ask me to remember any number. Our conversation jus